In [2]:
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException

from difflib import SequenceMatcher
from bs4 import BeautifulSoup
############################################################
import os, csv
import time


class search_save_CSV:
    def __init__(self, i=0):
        
        header = [\
            "user-agent=Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3", \
            "user-agent=Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36", \
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome", \
        ]
        
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument(header[i])
        options.add_argument('Accept-Charset=utf-8')
        
        self.driver = webdriver.Chrome("./chromedriver.exe", options=options)
        self.driver.implicitly_wait(5)
        
    def getKeyword_andOpen(self, search_text):
        self.driver.get("https://m.map.naver.com/")
        
        self.search_ = search_text
        self.driver.find_element_by_class_name("Nsearch._searchKeywordView._searchGuide").click()
        
        self.driver.find_element_by_class_name("Nbox_input_text._search_input").send_keys(self.search_)
        self.driver.find_element_by_class_name("_formSearch").submit()
        
    def errorCheck(self):
        # print(self.driver.find_element_by_class_name('interstitial-wrapper'))
        try:
            self.driver.find_element_by_class_name('interstitial-wrapper')
            print("error-reload")
            return True
        except NoSuchElementException:
            return False
        
    def search_and_save(self):
        searchList = self.driver.find_elements_by_class_name("_item._lazyImgContainer")
        choicePer, choiceIndex = 0, -1
        for i, element in enumerate(searchList):
            # print(i, ":", element.get_attribute("data-title"))    
            ratio = self.getMatchPercent(element.get_attribute("data-title"), self.search_)
            print("{0:.1f}%".format(ratio))
            
            if ratio < 50:
                pass
            else:
                if choicePer < ratio:
                    choicePer = ratio
                    choiceIndex = i
        if choiceIndex == -1:
            print("일치결과 없음")
            return False
        else:
            print("선택 index {0} : {1} 과의 일치율 {2:.1f}".format(choiceIndex, self.search_, choicePer))
            self.index = choiceIndex
            return True
        
    def get_menu_list(self): 
        elements = self.driver.find_elements_by_class_name("sp_map.btn_price._linkMenu")
        try:
            elements[self.index].click()
        except Exception:
            "가격정보 없음"
            return False
        time.sleep(5)
        
        temp = str(self.driver.current_url)
        temp = temp[:-4] + 'menu/list'
        print(temp)
        self.driver.get(temp)
        
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        elements = soup.find_all("li", class_="_1jXsR")
        menuName = soup.find_all("span", class_="_1zr5s")
        menuPrice = soup.find_all("div", class_="_1cQ4a")

        for i, menu in enumerate(menuName):
            print(i, menu.text, ":", menuPrice[i].text)
            # menuPrice[i].text = menuPrice[i].text.replace(",", "")
        self.name = menuName
        self.price = menuPrice
        return True
        
    def save_to_csv(self, pk):
        saveFileName = os.path.abspath("../data/price.csv")
        if not os.path.exists(saveFileName):
            file = open(saveFileName, "w", newline="", encoding='utf-8')
            newWriter = csv.writer(file)
            newWriter.writerow(["관리번호", "메뉴이름", "가격"])
            file.close()
        file = open(saveFileName, "a", newline="", encoding='utf-8')
        writer = csv.writer(file)
        prices = [i.text for i in self.price]
        for i, menu in enumerate(self.name):
            row = [str(pk), menu.text, prices[i].replace(',', '')]
            writer.writerow(row)
        print("save success")
        file.close()      
        
        
    def getMatchPercent(self, one, two):
        s = SequenceMatcher(None, one, two)
        return s.ratio() * 100
    
    def close_driver(self):
        self.driver.quit()

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("coffee-price").setMaster("local[*]")
sc = SparkContext(conf=conf).getOrCreate()

In [4]:
import os

data_path = os.path.join("..", "data", "data.csv")
raw_data = sc.textFile(data_path)
raw_data_header = raw_data.take(1)[0]

In [5]:
headerList = str(raw_data_header).split(",") # 관리번호 = PK
need_column = ['관리번호', '영업상태구분코드', '영업상태명', '소재지전화',\
        '소재지면적','소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점',\
        '업태구분명', '좌표정보(X)', '좌표정보(Y)', '시설총규모', '홈페이지']
need_column_index = []
column_list = []
for i, value in enumerate(headerList):
    if str(value) in need_column:
        need_column_index.append(i)
        column_list.append(str(value))

In [6]:
data = raw_data.filter(lambda line: line!=raw_data_header) \
    .map(lambda line: line.split(",")) \
    .filter(lambda tokens: tokens[24] == '커피숍' and tokens[7] =='영업/정상') \
    .map(lambda tokens: [tokens[i] for i in need_column_index]) \
    .filter(lambda tokens: tokens[5].split()[0] == '경기도' or tokens[5].split()[0] == '충청북도')

In [7]:
elements = data.take(100000)
print(len(elements))

16000


In [8]:
elements = data.take(100000)
dongList = list()
num = 0
search = search_save_CSV(num)

for i, element in enumerate(elements):
    if i < 1195:
        continue
    print(i, "번째.")
#     if i%10 == 9:
#         search.close_driver()
#         time.sleep(1)
#         num = num + 1
#         search = search_save_CSV(num % 3)
    for i in range(len(column_list)):
        print("[%s]" % i, column_list[i], ":", element[i])
    continue_list = \
    ["스타벅스", "이디야", "투썸", "투썸플레이스", "엔젤리너스", "커피빈", "구스토", "달콤커피", "메가커피", \
    "바나프레소", "빽다방", "셀렉토", "요거프레소", "주커피", "드롭탑", "띠아모", "카페베네", "봄봄", \
    "커핀그루나루", "커피마마", "커피에 반하다", "탐앤탐스", "폴 바셋", "할리스", "파스쿠찌", "쥬씨"]
    
    name_list = str(element[7]).split()
    flag = False
    for comp in name_list:
        for li in continue_list:
            if li in comp:
                flag = True
                break
        if flag:
            break
    if flag: # 프랜차이즈의 경우는 넘어간다.
        continue
    
    address = str(element[5]).split()
    try:
        dongList.append(address[2])
        print("************************************************")
        search_keyword = address[2] + " " + element[7]
        
        search.getKeyword_andOpen(search_keyword)
    
        
        # check error page
        while search.errorCheck():
            time.sleep(10)
            search.close_driver()
            num = num + 1
            search = search_save_CSV(num % 3)
            search.getKeyword_andOpen(search_keyword)
            
            
        flag_ = search.search_and_save()
        if flag_:
            if search.get_menu_list():
                time.sleep(1)
                search.save_to_csv(element[0])
    except Exception:
        continue
search.close_driver()

1195 번째.
[0] 관리번호 : 3760000-104-2018-00128
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 35.85
[5] 소재지전체주소 : 경기도 수원시 권선구 탑동 835-4번지 1층
[6] 도로명전체주소 : 경기도 수원시 권선구 탑동로3번길 13  1층 (탑동)
[7] 사업장명 : 빽다방 수원탑동점
[8] 최종수정시점 : 20181113205036
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.26887653295994
[11] 좌표정보(Y) : 126.9744781742452
[12] 시설총규모 : 35.85
[13] 홈페이지 : None
1196 번째.
[0] 관리번호 : 3750000-104-2018-00092
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 53.56
[5] 소재지전체주소 : 경기도 수원시 장안구 천천동 524-3번지 로얄프라자 102호
[6] 도로명전체주소 : 경기도 수원시 장안구 덕영대로535번길 38  로얄프라자 1층 102호 (천천동)
[7] 사업장명 : 메가엠지씨커피 수원천천점
[8] 최종수정시점 : 20181019111328
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.29711959316762
[11] 좌표정보(Y) : 126.98236813234611
[12] 시설총규모 : 53.56
[13] 홈페이지 : None
************************************************
일치결과 없음
1197 번째.
[0] 관리번호 : 3760000-104-2018-00133
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 39.52
[5] 소재지전체주소 : 경기도 수원시 권선구 금곡동 1117-5번지
[6] 도로명전체주소 : 경기도 수원시 권선구 금곡로

37.5%
35.3%
일치결과 없음
1210 번째.
[0] 관리번호 : 5610000-104-2018-00145
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 127.89
[5] 소재지전체주소 : 경기도 수원시 영통구 이의동 1216-4번지
[6] 도로명전체주소 : 경기도 수원시 영통구 웰빙타운로36번길 46-113  1층 (이의동)
[7] 사업장명 : 베이향
[8] 최종수정시점 : 20180917091513
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.30907539162789
[11] 좌표정보(Y) : 127.05168919975382
[12] 시설총규모 : 127
[13] 홈페이지 : None
************************************************
61.5%
선택 index 0 : 영통구 베이향 과의 일치율 61.5
https://m.place.naver.com/restaurant/1281996150/menu/list
0 아메리카노 : 4,500원
1 로얄밀크티 : 6,000원
2 카페라테 : 5,000원
3 바닐라라떼 : 5,500원
4 카페모카 : 5,500원
5 카라멜마끼아또 : 6,000원
6 카푸치노 : 6,000원
7 아포카토 : 6,000원
8 허브티 : 5,000원
9 홍차(잎차) : 7,000원
10 티카푸치노 : 7,000원
11 녹차라떼,초코라떼 : 5,000원
12 스무디 : 5,500원
13 흑당라떼 : 5,500원
14 당근케잌 : 5,400원
save success
1211 번째.
[0] 관리번호 : 3750000-104-2018-00084
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 251 3550
[4] 소재지면적 : 374.26
[5] 소재지전체주소 : 경기도 수원시 장안구 파장동 469-4번지 1 2층
[6] 도로명전체주소 : 경기도 수원시 장안구 경수대로 

75.0%
선택 index 0 : 팔달구 남수동452 과의 일치율 75.0
https://m.place.naver.com/restaurant/1005987321/menu/list
0 라떼 : 4,500원
1 마카롱 : 2,600원
2 아메리카노 : 4,000원
3 바닐라 라떼 : 5,000원
4 카라멜 마끼아또 : 변동
5 머랭쿠키 : 변동
6 스콘 : 변동
save success
1225 번째.
[0] 관리번호 : 3770000-104-2018-00117
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 167.52
[5] 소재지전체주소 : 경기도 수원시 팔달구 신풍동 221-21번지 1층
[6] 도로명전체주소 : 경기도 수원시 팔달구 신풍로 32  1층 (신풍동)
[7] 사업장명 : 카페은연
[8] 최종수정시점 : 20180927130252
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.2837948364041
[11] 좌표정보(Y) : 127.01475804347561
[12] 시설총규모 : 167.52
[13] 홈페이지 : None
************************************************
66.7%
선택 index 0 : 팔달구 카페은연 과의 일치율 66.7
https://m.place.naver.com/restaurant/1193347963/menu/list
0 아메리카노 : 4,000원
1 은연라떼 : 5,500원
2 당근케이크 : 6,000원
3 콘파냐 : 5,500원
4 레몬티 : 4,500원
save success
1226 번째.
[0] 관리번호 : 5610000-104-2018-00137
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 158.20
[5] 소재지전체주소 : 경기도 수원시 영통구 하동 1024번지
[6] 도로명전체주소 : 경기도 수원시 영통구 광교호수로

80.0%
선택 index 0 : 영통구 시나본 갤러리아광교 과의 일치율 80.0
https://m.place.naver.com/restaurant/1433999626/menu/list
0 아메리카노 : 3,500원
1 클래식본 : 4,500원
2 시나본 콤보 : 7,600원
save success
1241 번째.
[0] 관리번호 : 3760000-104-2020-00013
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 293 8497
[4] 소재지면적 : 490.42
[5] 소재지전체주소 : 경기도 수원시 권선구 오목천동 266-3번지
[6] 도로명전체주소 : 경기도 수원시 권선구 서수원로 245  1~3층 (오목천동)
[7] 사업장명 : 파스쿠찌 수원권선DT점
[8] 최종수정시점 : 20200217192819
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.24850891708974
[11] 좌표정보(Y) : 126.96664805354202
[12] 시설총규모 : 490.42
[13] 홈페이지 : None
1242 번째.
[0] 관리번호 : 3770000-104-2020-00012
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 02 5819090
[4] 소재지면적 : 49.42
[5] 소재지전체주소 : 경기도 수원시 팔달구 인계동 1036-7번지 수원인계 리슈빌S
[6] 도로명전체주소 : 경기도 수원시 팔달구 인계로138번길 8  수원인계 리슈빌S 1층 103호 (인계동)
[7] 사업장명 : 엔파체커피(enpache coffee)
[8] 최종수정시점 : 20200210110554
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.26676519010548
[11] 좌표정보(Y) : 127.03243374030525
[12] 시설총규모 : 49.42
[13] 홈페이지 : None
***************************************

일치결과 없음
1257 번째.
[0] 관리번호 : 3770000-104-2020-00047
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 69.19
[5] 소재지전체주소 : 경기도 수원시 팔달구 매산로1가 18번지 AK플라자
[6] 도로명전체주소 : 경기도 수원시 팔달구 덕영대로 924  AK플라자 6층 (매산로1가)
[7] 사업장명 : 타이거슈가 수원AK점
[8] 최종수정시점 : 20200325164627
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.266100573245744
[11] 좌표정보(Y) : 126.9998641660733
[12] 시설총규모 : 69.19
[13] 홈페이지 : None
************************************************
일치결과 없음
1258 번째.
[0] 관리번호 : 3760000-104-2020-00043
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 수원시 권선구 평동 179번지
[6] 도로명전체주소 : 경기도 수원시 권선구 평동로79번길 45  117호 (평동)
[7] 사업장명 : 117하이드아웃(HIDEOUT)
[8] 최종수정시점 : 20200309143254
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : None
[11] 좌표정보(Y) : None
[12] 시설총규모 : 0
[13] 홈페이지 : None
************************************************
일치결과 없음
1259 번째.
[0] 관리번호 : 3750000-104-2020-00004
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 070 75434749
[4] 소재지면적 : 33.00
[5] 소재지전체주소 : 경기도 수원시 장안구 정자동 877-4

...

71.4%
38.1%
11.1%
0.0%
21.1%
19.0%
30.8%
11.1%
11.1%
26.7%
0.0%
22.2%
30.8%
20.0%
0.0%
26.7%
33.3%
13.8%
0.0%
11.8%
33.3%
0.0%
0.0%
0.0%
14.3%
33.3%
0.0%
12.5%
0.0%
0.0%
0.0%
0.0%
0.0%
11.4%
선택 index 0 : 분당구 운중동커피 과의 일치율 71.4
https://m.place.naver.com/restaurant/36994785/menu/list
0 임실요거트1잔 : 3,000원
save success
1702 번째.
[0] 관리번호 : 3800000-104-2015-00031
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 79.68
[5] 소재지전체주소 : 경기도 성남시 중원구 상대원동 513-14번지 B106호
[6] 도로명전체주소 : 경기도 성남시 중원구 둔촌대로 484  B106호 (상대원동)
[7] 사업장명 : 카페밈스
[8] 최종수정시점 : 20190402115924
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.43106165545304
[11] 좌표정보(Y) : 127.17061576960805
[12] 시설총규모 : 95.4
[13] 홈페이지 : None
************************************************
66.7%
선택 index 0 : 중원구 카페밈스 과의 일치율 66.7
https://m.place.naver.com/restaurant/1597925986/menu/list
0 아메리카노 : 2,300원
1 에스프레소 : 1,900원
2 콜드브루 : 3,000원
3 카푸치노 : 3,300원
4 카페라떼 : 3,300원
save success
1703 번째.
[0] 관리번호 : 3790000-104-2015-00034
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 

80.0%
선택 index 0 : 분당구 웍스프레소앳판교 과의 일치율 80.0
https://m.place.naver.com/restaurant/38331990/menu/list
0 에스프레소 : 3,000원
save success
1718 번째.
[0] 관리번호 : 3810000-104-2014-00195
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 715 5638
[4] 소재지면적 : 199.67
[5] 소재지전체주소 : 경기도 성남시 분당구 구미동 159번지 월드쇼핑 1층 101호일부
[6] 도로명전체주소 : 경기도 성남시 분당구 탄천상로151번길 20 (구미동  월드쇼핑 1층 101호일부)
[7] 사업장명 : (주)커피빈코리아 오리역애플플라자점
[8] 최종수정시점 : 20180223134411
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.3399855959003
[11] 좌표정보(Y) : 127.10694057997429
[12] 시설총규모 : 199.67
[13] 홈페이지 : None
1719 번째.
[0] 관리번호 : 3800000-104-2016-00030
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 25.31
[5] 소재지전체주소 : 경기도 성남시 중원구 금광동 4845번지 1층
[6] 도로명전체주소 : 경기도 성남시 중원구 산성대로476번길 6  1층 (금광동)
[7] 사업장명 : 투데이
[8] 최종수정시점 : 20191209142318
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.45366238866024
[11] 좌표정보(Y) : 127.16142718840996
[12] 시설총규모 : 25.31
[13] 홈페이지 : None
************************************************
28.6%
50.0%
37.5%
46.2%
50.0%
30.0%
50.0%
0.

https://m.place.naver.com/restaurant/37427552/menu/list
0 아메리카노 테이크아웃 : 2,000원
save success
1735 번째.
[0] 관리번호 : 3810000-104-2016-00069
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 702 1226
[4] 소재지면적 : 28.48
[5] 소재지전체주소 : 경기도 성남시 분당구 백현동 531번지 알파리움 타워1동 2층 229호
[6] 도로명전체주소 : 경기도 성남시 분당구 대왕판교로606번길 10  타워1동 2층 229호 (백현동  알파리움)
[7] 사업장명 : 띠아모플라워 커피베이판교알파돔시티점
[8] 최종수정시점 : 20160428171723
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.39542424069859
[11] 좌표정보(Y) : 127.10817067684863
[12] 시설총규모 : 28.48
[13] 홈페이지 : None
1736 번째.
[0] 관리번호 : 3790000-104-2016-00082
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 722 1590
[4] 소재지면적 : 81.28
[5] 소재지전체주소 : 경기도 성남시 수정구 창곡동 544-3번지 1층
[6] 도로명전체주소 : 경기도 성남시 수정구 위례서일로1길 22-8  1층 (창곡동)
[7] 사업장명 : 블레스롤(Bless Roll)
[8] 최종수정시점 : 20161010201914
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.467451845742026
[11] 좌표정보(Y) : 127.13858519108851
[12] 시설총규모 : 81.28
[13] 홈페이지 : None
************************************************
33.3%
일치결과 없음
1737 번째.
[0] 관리번호 : 3800000-104-2016-

84.6%
선택 index 0 : 수정구 모노치즈 남한산성역점 과의 일치율 84.6
https://m.place.naver.com/restaurant/1269418060/menu/list
0 그릴드치즈 클래식 : 2,000원
1 그릴드치즈 햄치즈 : 2,500원
2 그릴드치즈 고구마 : 2,500원
3 그릴드치즈 할라피뇨 : 2,700원
4 그릴드치즈 카야치즈 : 2,900원
save success
1752 번째.
[0] 관리번호 : 3790000-104-2016-00035
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 40.14
[5] 소재지전체주소 : 경기도 성남시 수정구 신흥동 5493-1번지 1층 일부 104호
[6] 도로명전체주소 : 경기도 성남시 수정구 수정로 202  1층 (신흥동)
[7] 사업장명 : 더 벤티
[8] 최종수정시점 : 20160420153339
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.44375041444816
[11] 좌표정보(Y) : 127.14224158445924
[12] 시설총규모 : 40.14
[13] 홈페이지 : None
************************************************
35.3%
37.5%
40.0%
40.0%
35.3%
31.6%
일치결과 없음
1753 번째.
[0] 관리번호 : 3810000-104-2016-00211
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 31.41
[5] 소재지전체주소 : 경기도 성남시 분당구 수내동 29번지 양지마을 주상복합상가 603 1층 148호
[6] 도로명전체주소 : 경기도 성남시 분당구 내정로173번길 11  1층 148호 (수내동  양지마을 주상복합상가 603)
[7] 사업장명 : 마리앤마사
[8] 최종수정시점 : 20160826135532
[9] 업태구분명 : 커피숍
[10] 좌표

KeyboardInterrupt: 